In [ ]:
%pylab inline

In [ ]:
import numpy as np

class Asinh(object):
    def fit(self, y):
        tht = np.linspace(.001, 1.0, 100)
        
        ftht = [self._neglog(y, x) for x in tht]

        ls = tht[np.argmax(ftht)]
        
        self.optim_ = ls
        
    def transform(self, y):
        return self._f(y, self.optim_)
        
    
    def _f(self, y, theta):
        if abs(theta) < 1e-5:
            return y
        else:
            return np.arcsinh(theta*y)/theta

    def _neglog(self, y, theta):
        ft = self._f(y, theta)
        val = - .5 * np.log(ft.dot(ft-ft.mean())) \
              - .5 * np.mean(np.log(1+(theta*y)**2))
            
        return val

In [ ]:
import xarray as xr

In [ ]:
q1 = xr.open_dataset("wd/ml/Y.nc").q1
q1 = q1-q1.mean(['x', 'time'])

q1 = q1.isel(time=slice(-50,None))

# data for distribution
y = q1.values.ravel()
yrand = np.random.choice(y, 20000, replace=False)

In [ ]:
import scipy.stats as stats

You can see that the distribution of Q1 has extremely fat tails

In [ ]:
stats.probplot(q1.values.ravel(), plot=pyplot)

In [ ]:
theta= .16
y= q1.values.ravel()

y_normalized = np.arcsinh(theta*y)/theta
stats.probplot(y_normalized, plot=pyplot)


In [ ]:
mod = Asinh()
mod.fit(y)

In [ ]:
mod.optim_

In [ ]:
mod._neglog(y, .16)

In [ ]:
mod._neglog(y, -.001)

In [ ]:
y_normalized = mod.transform(y)
stats.probplot(y_normalized, plot=pyplot);

What about this Tunkey-Lambda ccf plot

In [ ]:
stats.ppcc_plot(yrand, -1, 1, plot=plt);

the maximum value is given by

In [ ]:
lam_max = stats.ppcc_max(yrand)
lam_max

In [ ]:
y_transformed = np.sign(yrand) * np.abs(yrand)**lam_max